In [1]:
import keras
from keras import models
from keras import layers
from keras import optimizers
import os, shutil
import utils
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from ML_algorithms import *
from numpy import array
from numpy import hstack
import numpy as np

Using TensorFlow backend.


## 1. Image data

In [ ]:
train_dir = "data_files/Cactus_Image/training_set"
test_dir = "data_files/Cactus_Image/testing_set"

In [ ]:
def keras_cnn(n_neurons=32, n_layers=3, filter_size=(3, 3), activation="relu", 
               input_shape =(64,64,3), max_pooling=(2,2), dense_layer=128, 
               loss="binary_crossentropy",optimizer="adam",metrics="acc"):
    # NOTE: always alter the input_shape to the specific input shape off the problem.

    model = models.Sequential()
    model.add(layers.Conv2D(n_neurons, filter_size, activation=activation,
                           input_shape =input_shape))
    model.add(layers.MaxPooling2D(max_pooling))
    for num in range(n_layers-2):
        model.add(layers.Conv2D(n_neurons, filter_size, activation=activation))
        model.add(layers.MaxPooling2D(max_pooling))
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_layer, activation=activation))
    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=[metrics])
    
    return model


In [ ]:
model = keras_cnn() 
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64,64),
    batch_size=20,
    class_mode="binary")

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=20,
    class_mode="binary")

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=250,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50)

## 2. Timeseries Data

In [2]:
X_train, X_test, y_train, y_test = utils.get_timeseries_dataset(cnn_or_lstm=True)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/jovyan/work/2_Semester/Deep Learning/DLProject/utils.py:192: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X_train = X_train.as_matrix().reshape((len(X_train), 15))
/home/jovyan/work/2_Semester/Deep Learning/DLProject/utils.py:193: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  y_train = y_train.as_matrix().reshape((len(y_train), 1))
/home/jovyan/work/2_Semester/Deep Learning/DLProject/utils.py:194: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X_test = X_test.as_matrix().reshape((len(X_test), 15))
/home/jovyan/work/2_Semester/Deep Learning/DLProject/utils.py:195: FutureWarning: Method .as_matrix will be removed in a future version. 

In [3]:
model = keras_cnn_conv1D()

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test))

Train on 1047 samples, validate on 262 samples
Epoch 1/10
1047/1047 [==============================] - 0s 47us/step - loss: 1.5948 - acc: 0.6877 - val_loss: 3.9170 - val_acc: 0.5649
Epoch 2/10
1047/1047 [==============================] - 0s 39us/step - loss: 2.0018 - acc: 0.6504 - val_loss: 0.7379 - val_acc: 0.7481
Epoch 3/10
1047/1047 [==============================] - 0s 36us/step - loss: 0.5376 - acc: 0.7650 - val_loss: 0.6240 - val_acc: 0.7214
Epoch 4/10
1047/1047 [==============================] - 0s 39us/step - loss: 0.4997 - acc: 0.7813 - val_loss: 0.5454 - val_acc: 0.7519
Epoch 5/10
1047/1047 [==============================] - 0s 41us/step - loss: 0.4875 - acc: 0.7899 - val_loss: 0.5512 - val_acc: 0.7481
Epoch 6/10
1047/1047 [==============================] - 0s 38us/step - loss: 0.4831 - acc: 0.7947 - val_loss: 0.5525 - val_acc: 0.7557
Epoch 7/10
1047/1047 [==============================] - 0s 40us/step - loss: 0.4783 - acc: 0.7985 - val_loss: 0.6010 - val_acc: 0.7595
Epoch 8/

## 2. Titanic Data 
### 2.1 conv1D

In [31]:
X_train, X_test, y_train, y_test = utils.get_titanic_dataset(cnn_or_lstm=True)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/home/jovyan/work/2_Semester/Deep Learning/DLProject/utils.py:97: FutureWarning: Method .as_matrix will be removed in a futu

In [21]:
X_train.shape

(1047, 1, 36)

In [33]:
model = keras_cnn_conv1D(filters=6, input_shape=(1,36), pool_size=1,kernel_size=1)

In [34]:
model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 1047 samples, validate on 262 samples
Epoch 1/100
1047/1047 [==============================] - 1s 588us/step - loss: 3.4329 - f1: 0.5513 - val_loss: 4.4179 - val_f1: 0.5232
Epoch 2/100
1047/1047 [==============================] - 0s 39us/step - loss: 3.8007 - f1: 0.5132 - val_loss: 4.1173 - val_f1: 0.5361
Epoch 3/100
1047/1047 [==============================] - 0s 40us/step - loss: 3.3062 - f1: 0.5443 - val_loss: 2.6152 - val_f1: 0.5212
Epoch 4/100
1047/1047 [==============================] - 0s 48us/step - loss: 2.4082 - f1: 0.5572 - val_loss: 2.1770 - val_f1: 0.5208
Epoch 5/100
1047/1047 [==============================] - 0s 39us/step - loss: 3.3367 - f1: 0.3584 - val_loss: 5.6869 - val_f1: 0.0628
Epoch 6/100
1047/1047 [==============================] - 0s 37us/step - loss: 4.8919 - f1: 0.2222 - val_loss: 4.7331 - val_f1: 0.2949
Epoch 7/100
1047/1047 [==============================] - 0s 38us/step - loss: 4.7557 - f1: 0.3188 - val_loss: 4.7294 - val_f1: 0.2949
Epoch 8/100
10

1047/1047 [==============================] - 0s 49us/step - loss: 0.5837 - f1: 0.5752 - val_loss: 0.6463 - val_f1: 0.5510
Epoch 62/100
1047/1047 [==============================] - 0s 50us/step - loss: 0.8744 - f1: 0.4919 - val_loss: 2.2186 - val_f1: 0.0204
Epoch 63/100
1047/1047 [==============================] - 0s 50us/step - loss: 1.9650 - f1: 0.3774 - val_loss: 1.3039 - val_f1: 0.5380
Epoch 64/100
1047/1047 [==============================] - 0s 44us/step - loss: 1.4483 - f1: 0.5154 - val_loss: 0.8487 - val_f1: 0.6128
Epoch 65/100
1047/1047 [==============================] - 0s 47us/step - loss: 0.6795 - f1: 0.6581 - val_loss: 0.5460 - val_f1: 0.6457
Epoch 66/100
1047/1047 [==============================] - 0s 49us/step - loss: 0.4823 - f1: 0.6754 - val_loss: 0.4867 - val_f1: 0.6879
Epoch 67/100
1047/1047 [==============================] - 0s 46us/step - loss: 0.4707 - f1: 0.6921 - val_loss: 0.4770 - val_f1: 0.6986
Epoch 68/100
1047/1047 [==============================] - 0s 45us/st

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 1, 6)              222       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 6)              0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                350       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 51        
Total params: 623
Trainable params: 623
Non-trainable params: 0
_________________________________________________________________


### 2.2 conv2D

In [7]:
X_train, X_test, y_train, y_test = utils.get_titanic_dataset(cnn_conv2d=True)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [8]:
model = keras_cnn(filter_size=(1,1), input_shape=(1, 36, 1), max_pooling=(1,1))

In [9]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test))

Train on 1047 samples, validate on 262 samples
Epoch 1/10
1047/1047 [==============================] - 0s 439us/step - loss: 0.6271 - f1: 0.4542 - val_loss: 0.6039 - val_f1: 0.0889
Epoch 2/10
1047/1047 [==============================] - 0s 143us/step - loss: 0.5197 - f1: 0.6208 - val_loss: 0.5316 - val_f1: 0.7017
Epoch 3/10
1047/1047 [==============================] - 0s 132us/step - loss: 0.4850 - f1: 0.6872 - val_loss: 0.5131 - val_f1: 0.7045
Epoch 4/10
1047/1047 [==============================] - 0s 139us/step - loss: 0.4550 - f1: 0.7123 - val_loss: 0.4936 - val_f1: 0.7233
Epoch 5/10
1047/1047 [==============================] - 0s 131us/step - loss: 0.4496 - f1: 0.7088 - val_loss: 0.4980 - val_f1: 0.7148
Epoch 6/10
1047/1047 [==============================] - 0s 123us/step - loss: 0.4402 - f1: 0.7119 - val_loss: 0.4740 - val_f1: 0.7268
Epoch 7/10
1047/1047 [==============================] - 0s 119us/step - loss: 0.4500 - f1: 0.7068 - val_loss: 0.4762 - val_f1: 0.7469
Epoch 8/10
1047

In [ ]:
#img_rows, img_cols = 1,36
#nb_filters = 1000
#pool_size = (1, 1)
#kernel_size = (1, 1)

In [ ]:
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten

model = Sequential()
model.add(Convolution2D(nb_filters, filter_size=(1,1),
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='mean_squared_error', optimizer='Nadam', metrics=['binary_accuracy'])

model.fit(X_train, y_train_onehot, epochs=30)

In [ ]:
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten

model = models.Sequential()
model.add(layers.Conv2D(32, filter_size=(1,1), activation="relu", input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Conv2D(32, filter_size=(1,1), activation="relu"))
model.add(layers.MaxPooling2D(pool_size))

model.add(layers.Flatten())
model.add(layers.Dense(dense_layer, activation=activation))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(loss='mean_squared_error', optimizer='Nadam', metrics=['binary_accuracy'])

model.fit(X_train, y_train_onehot, epochs=30)

In [ ]:
X_train = X_train.as_matrix().reshape((len(X_train), 15))
y_train = y_train.as_matrix().reshape((len(y_train), 1))
X_test = X_test.as_matrix().reshape((len(X_test), 15))
y_test = y_test.as_matrix().reshape((len(y_test), 1))

train_dataset=hstack((X_train,y_train))
test_dataset=hstack((X_test,y_test))

def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

X_train, y_train = split_sequences(train_dataset, 3)
X_test, y_test = split_sequences(test_dataset, 3)

In [ ]:
def keras_cnn_conv1D(filters=64, n_layers=2, kernel_size=2, activation="relu", 
               input_shape =(3,15), pool_size=2, dense_layer=50, 
               loss="binary_crossentropy",optimizer="adam",metrics="acc"):

    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=pool_size))
    for num in range(n_layers-2):
        model.add(Conv1D(filters=filters, kernel_size=kernen_size, activation=activation, input_shape=input_shape))
        model.add(MaxPooling1D(pool_size=pool_size))    
    model.add(Flatten())
    model.add(Dense(dense_layer, activation=activation))
    model.add(Dense(1))
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metrics])
    
    return model